<a href="https://colab.research.google.com/github/rudevico/Gachon-AISTUDY/blob/main/UCI_raisin_Classifications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Introdction
* [Download sample data set: Raisin](https://archive.ics.uci.edu/dataset/850/raisin) (CINAR I., KOKLU M. and TASDEMIR S., 2020)[1]

UCI Macine Learning Repository의 Raisin(; 건포도) data set을 사용해서 5가지 타입의 분류를 수행합니다.  
또한 그 중에서도 우선 **Binary Classification**에 대해서만 수행합니다.

**types of Classification algorithms that we use**  
* #1. K-Nearest Neighbors; K-NN
* #2. Logistic Regression; LR
* #3. Decision Tree; DT
* #4. Random Forest; RF
* #5. Support Vector Machine; SVM

## 0. 1. How to Upload & Load file to google colab
[ref1. google colab에서 file 사용하기](https://teki.tistory.com/29)  
[ref2. colab에 csv 파일 불러오기](https://resultofeffort.tistory.com/60)  

다음 cell에서 google colab의 default working directory를 확인할 수 있다.

In [ ]:
import os

# in colab, default working directory is /content
print(os.popen('pwd').read())

/content



In [30]:
from google.colab import files

uploaded = files.upload()

Saving Raisin_Dataset.xlsx to Raisin_Dataset.xlsx


# 1. K-Nearest Neighbors; K-NN




[manual of pandas.read_excel](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html)

In [50]:
import pandas as pd

# df = pd.read_excel('./Raisin_Dataset.xlsx', header=None)
# df = pd.read_excel('./Raisin_Dataset.xlsx', header=1)
raisin = pd.read_excel('./Raisin_Dataset.xlsx', header=0) # cf. header parameter default = 0
raisin.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


In [52]:
print(raisin.shape) # cf. 900 rows, 8 columns

(900, 8)


해당 실습에서는 Classification을 진행하므로 다음과 같이 구분할 수 있겠다.  
* 900개의 **data instances**
* 7개의 **features**(Area, MajorAxisLength, ..., Perimeter) -> 수학에서의 *independent variables*
* 1개의 **label**(Class) -> 수학에서의 *dependent variable*

또한 다음 cell과 같이 Class에 어떤 종류가 있는지 확인할 수 있다.

In [42]:
print(raisin['Class'].unique()) # cf. pandas dataframe에서의 unique 출력 방법(1)
print(pd.unique(raisin['Class'])) # cf. pandas dataframe에서의 unique 출력 방법(2)

['Kecimen' 'Besni']
['Kecimen' 'Besni']


In [43]:
print(raisin.to_numpy()) # cf. default, no include header

[[87524 442.2460114 253.291155 ... 0.758650579 1184.04 'Kecimen']
 [75166 406.690687 243.0324363 ... 0.68412957 1121.786 'Kecimen']
 [90856 442.2670483 266.3283177 ... 0.637612812 1208.575 'Kecimen']
 ...
 [99657 431.7069809 298.8373229 ... 0.741098519 1292.828 'Besni']
 [93523 476.3440939 254.1760536 ... 0.658798253 1258.548 'Besni']
 [85609 512.0817743 215.2719758 ... 0.632019963 1272.862 'Besni']]


In [53]:
raisin_input = raisin[['Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity', 'ConvexArea', 'Extent','Perimeter']].to_numpy()
raisin_target = raisin['Class'].to_numpy()
print(raisin_input[:3]) # standardization 필요한 것을 확인할 수 있음
print(raisin_target[:3])

[[8.75240000e+04 4.42246011e+02 2.53291155e+02 8.19738392e-01
  9.05460000e+04 7.58650579e-01 1.18404000e+03]
 [7.51660000e+04 4.06690687e+02 2.43032436e+02 8.01805234e-01
  7.87890000e+04 6.84129570e-01 1.12178600e+03]
 [9.08560000e+04 4.42267048e+02 2.66328318e+02 7.98353619e-01
  9.37170000e+04 6.37612812e-01 1.20857500e+03]]
['Kecimen' 'Kecimen' 'Kecimen']


이제 다음 작업을 수행한다.  
* data set을 train set과 test으로 나누기
* standardization

In [58]:
from sklearn.model_selection import train_test_split

# cf. default, suffle parameter=True, test_size=0.25, train_size=1-test_size
train_input, test_input, train_target, test_target = train_test_split(
    raisin_input, raisin_target
)

print(train_target.shape, test_target.shape) # cf. split ratio 0.75 : 0.25
print(test_target[:10]) # cf. shuffle

(675,) (225,)
['Kecimen' 'Kecimen' 'Kecimen' 'Besni' 'Kecimen' 'Kecimen' 'Kecimen'
 'Besni' 'Kecimen' 'Kecimen']


In [60]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

print(train_scaled[:3]) # cf. standardization

[[ 0.35914333  0.40287916  0.38402173  0.40621766  0.32891392 -0.31076478
   0.37061037]
 [-0.29970586 -0.51698231  0.2045911  -0.89639823 -0.30466716  0.59254853
  -0.34254713]
 [ 0.38761164 -0.01874484  1.1790615  -1.12700226  0.3754609   0.33606188
   0.31542059]]


이제 다음 작업을 수행한다.  
* Model training(fitting) by `fit()`
* Model의 accuracy 확인 by `score()`

In [62]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier()
kn.fit(train_input, train_target)

# underfitting이라고 판단할 수 있겠다.
print(kn.score(train_input, train_target))
print(kn.score(test_input, test_target))

0.8637037037037038
0.8533333333333334


이제 다음 작업을 수행한다.  
* Solve the underfitting

> fix, solve, handle, deal with 등의 용어를 사용할 수 있다.  
  * [Link](https://www.linkedin.com/advice/0/how-can-you-fix-underfitting-deep-learning-optimization-jxjae)
  How can you **fix** underfitting in deep learning optimization?
  * [Link](https://allcloud.io/blog/how-to-solve-underfitting-and-overfitting-data-models/)
  How to **Solve** Underfitting and Overfitting Data Models
  * [Link](https://deepchecks.com/question/how-do-we-handle-overfitting-and-underfitting-in-a-machine-learning-model/) How do we **handle** overfitting and underfitting in a machine learning model?
  * [Link](https://medium.com/@akshaykr.sharma19966/steps-to-deal-with-overfitting-and-underfitting-on-image-data-using-image-augmentation-87848ab06351)
  Steps to **deal with** Overfitting and Underfitting on Image Data using Image Augmentation.

이때 underfitting의 원인에 따라 해결을 위한 접근방식을 다르게 취해야 한다.  
underfitting의 원인은 다음과 같이 추정할 수 있다.  

**common causes of underfitting**
  * 적은 데이터 수
  * 작은 복잡성
  * ... 이어서 영어로 쓰기

# References
[1] CINAR I., KOKLU M. and TASDEMIR S., (2020), Classification of Raisin Grains Using Machine Vision and Artificial Intelligence Methods. Gazi Journal of Engineering Sciences, vol. 6, no. 3, pp. 200-209, December, 2020.